In [ ]:
import pandas as pd
import numpy as np
import math
import time
import os

In [ ]:
LARGE_DATASET = False

In [ ]:
if LARGE_DATASET:
    movie_path = "ml-latest/movies.csv"
    tags_path = "ml-latest/tags.csv"
    ratings_path = "ml-latest/ratings.csv"
    movie_profiles_path = "/media/karol/E8B625B3B6258364/calculated/movie_profiles"
    user_profiles_path = "/media/karol/E8B625B3B6258364/calculated/user_profiles/"
    movie_matrix_path = "/media/karol/E8B625B3B6258364/calculated/movie_matrix"
else:
    movie_path = "ml-latest-small/movies.csv"
    tags_path = "ml-latest-small/tags.csv"
    ratings_path = "ml-latest-small/ratings.csv"
    movie_profiles_path = "/media/karol/E8B625B3B6258364/calculated-small/movie_profiles"
    user_profiles_path = "/media/karol/E8B625B3B6258364/calculated-small/user_profiles/"

In [ ]:
def read_user_profile(userId):
    directory = user_profiles_path + str(userId // 1000) + "/" + str(userId)
    user_profile = pd.read_csv(directory,  header=None, usecols=[1,2,3])
    return user_profile.rename(columns={1:"tag",2:"u_weight",3:"userId"})

def get_movie_profile(movieId):
    return movie_profiles[movie_profiles['movieId'] == movieId]

# Predict user's rating on test data

Finally a method fast enough to process large dataset

In [ ]:
def get_movie_profile_matrix(movie_indices):
    if len(movie_indices) == 1:
        mid = movie_indices.iloc[0]
        return get_movie_profile(mid).drop(['movieId'],1).rename(columns={"m_weight": mid})
    
    matrix_left = get_movie_profile_matrix(movie_indices[:len(movie_indices)//2])
    matrix_right = get_movie_profile_matrix(movie_indices[len(movie_indices)//2:])
    matrix = pd.merge(matrix_left,matrix_right , on='tag', how='outer', sort=False)
    return matrix

def get_similarity(userId, movies_df):
    i = read_user_profile(userId)
    i['u_weight'] = i['u_weight'] / np.sqrt(np.square(i['u_weight']).sum())
    similarity = pd.merge(i, movies_df, on='tag', how='outer', sort=False)
    return similarity.iloc[:,3:].mul(i['u_weight'],0).sum(0)

In [ ]:
mf = open(movie_profiles_path, 'r')
movie_profiles = pd.read_csv(mf).rename(columns={'weight':'m_weight'})

movies = movie_profiles['movieId'].drop_duplicates()

In [ ]:
movie_matrix = get_movie_profile_matrix(movies)

In [ ]:
ratings_data = pd.read_csv(ratings_path)

In [ ]:
start = time.time()
for userId in ratings_data['userId'].drop_duplicates():
    if userId != 469:
        f = open("calculated/content_based_recommendations/" + str(userId), "w")
        recomendations = pd.DataFrame()
        recomendations['movie'] = get_similarity(userId,movie_matrix).sort_values(ascending=False).index
        recomendations.to_csv(f, header=None ,index=False)
print(time.time() - start)

# Small Dataset processed in less than 4 seconds



in big dataset every user is calculated in less than second

TODO:

    ogarnąć dokumentacje, spostrzeżenia itp